In [9]:
import pandas as pd
import numpy as np
import re

df = pd.read_parquet('output/sentiments.parquet')
df.dropna(inplace=True)
df = df[(df['review'].apply(lambda x: re.match(r'^[()\[\]{}\'!,.: ]+$', x) is None)) & (df['review'].apply(lambda x: len(x) > 1))]
df.drop_duplicates(subset='review', ignore_index=True, inplace=True)
df.reset_index(drop=True, inplace=True)

df['sentiment'] = df['sentiment'].map({
    'Very Negative': 1,
    'Negative': 2,
    'Neutral': 3,
    'Positive': 4,
    'Very Positive': 5
})

df['game'] = df['game'].astype('category')

df

,topic,review,embedding,sentiment,game
0,gameplay mechanics,Excellent game mechanics hampered by really sh...,"[0.02064609, 0.058051355, -0.15955217, -0.0282...",2,1653100
1,gameplay mechanics,I love the narrator just talking in the backgr...,"[-0.01935661, 0.021597555, -0.18828672, -0.039...",4,1653100
2,gameplay mechanics,The game has received frequent and substantial...,"[0.030498626, 0.035088323, -0.17761268, -0.017...",4,1653100
3,gameplay mechanics,Heliotropism is a roguelike Tree simulator wit...,"[-0.005617, 0.06219709, -0.17552276, -0.042904...",4,1653100
4,narrator and audio,"the narrator is cool at first, then hyper anno...","[0.01688186, 0.010326986, -0.16718766, -0.0392...",2,1653100
...,...,...,...,...,...
91564,game mechanics,"The core gameplay is rather simple, but honest...","[0.027327502, 0.0755041, -0.17156118, -0.01077...",4,2300300
91565,music and sound,Хорошее музыкальное и звуковое сопровождение.,"[-0.035658363, -0.0012977426, -0.1295827, 0.02...",4,2300300
91566,music and sound,"I like the music, it's nice relaxing.","[-0.020432916, 0.03288259, -0.18042335, -0.001...",4,2300300
91567,price and value,The asking price might be a bit steep for what...,"[0.012826282, 0.04906256, -0.20521845, 0.02795...",3,2300300


In [10]:
games = pd.read_csv('./data/Game List - Final.csv')
games = games.dropna(subset=['SteamID']).rename(columns={'List (merge)': 'Game'})
games = games[games['Final decision'] == 'Yes']
games['SteamID'] = games['SteamID'].astype(int).astype(str).astype('category')

#df = games[['Game', 'SteamID']].merge(df, left_on='SteamID', right_on='game', how='inner')
#df

In [11]:
from src.embed import get_embedding
from sklearn.metrics.pairwise import cosine_similarity

citation_embeddings = np.vstack(df['embedding'].values)

def get_similar_citations(query, embeddings, threshold):    
    query_embedding = get_embedding([query])
    similarities = cosine_similarity(query_embedding, embeddings).flatten()
    indices = np.argwhere(similarities >= threshold)
    return indices.flatten()

In [12]:
import json

queries: dict = json.load(open('data/queries.json'))

results = {
    category: {
        q['name']: df.loc[get_similar_citations(q['name'], citation_embeddings, q['similarity_threshold'])]
        for q in cat_queries
    }
    for category, cat_queries in queries.items()
}

In [13]:
def get_table(results, agg_fn):
    df = pd.DataFrame()
    for category, queries in results.items():
        for query, citations in queries.items():
            df[(category, query)] = agg_fn(citations)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=['category', 'query'])
    return df

In [14]:
# count occurrences of each game for each query

get_table(results, lambda x: x['game'].value_counts())

category Gameplay Mechanics                                         \
query                Puzzle Survival Sandbox Building and Crafting   
game                                                                 
1593030                  54        0       5                    39   
1875060                  23        0       1                     8   
1372320                  12        0      62                     4   
2300300                   7        0       0                     0   
222730                    6        0      11                    10   
2109270                   5        0       1                     0   
1062090                   4       16      64                   539   
1575510                   3        0       0                     0   
914750                    2        0       0                     0   
1446000                   2        0       0                     0   
1612100                   1        0       0                     0   
2878100                   1        0       0                     0   
1346210                   0        0       0                     0   
1582290                   0        0       0                     0   
1390190                   0        0       5                    19   
1450250                   0        0       0                     1   
1539580                   0        0       0                     0   
1133120                   0        0       3                     0   
1028590                   0        0       0                     0   
1837210                   0        0       0                     0   
1647190                   0        0       0                     1   
1956670                   0        0       0                     0   
1919350                   0        0       0                     0   
1837330                   0        0       0                     0   
2211500                   0        0       0                     0   
2289390                   0        0       0                     0   
2294910                   0        0       0                     0   
1737200                   0        0       1                     0   
1653100                   0        0       0                     0   
2576600                   0        0       0                     0   
244710                    0        4       1                     2   
2635350                   0        0       1                     1   
2594480                   0        1       0                     0   
275100                    0        2       1                     2   
3059480                   0        0       0                     0   
382310                    0       13      26                   204   
2634610                   0        0       0                     0   
431180                    0        0       0                     0   
434000                    0        0       0                     0   
512460                    0        1       2                     0   
486310                    0        1       2                     1   
853550                    0        0      28                     7   
898890                    0        3       0                     1   
916900                    0        0       0                     0   
926990                    0        2       1                     1   

category                                                             \
query    Combat System and Action Economy and Management Simulation   
game                                                                  
1593030                         1                      6          4   
1875060                         1                      3          1   
1372320                         0                      0          0   
2300300                         0                      0          0   
222730                          0                      8          8   
2109270                         0                      0          0   


In [15]:
# average sentiment for each query

get_table(results, lambda x: x.groupby('game', observed=False)['sentiment'].mean())

category Gameplay Mechanics                                            \
query                Puzzle  Survival   Sandbox Building and Crafting   
game                                                                    
1028590                 NaN       NaN       NaN                   NaN   
1062090            3.000000  3.875000  3.843750              4.098330   
1133120                 NaN       NaN  4.000000                   NaN   
1346210                 NaN       NaN       NaN                   NaN   
1372320            4.333333       NaN  3.774194              4.250000   
1390190                 NaN       NaN  4.400000              4.000000   
1446000            4.000000       NaN       NaN                   NaN   
1450250                 NaN       NaN       NaN              3.000000   
1539580                 NaN       NaN       NaN                   NaN   
1575510            3.666667       NaN       NaN                   NaN   
1582290                 NaN       NaN       NaN                   NaN   
1593030            4.148148       NaN  3.800000              3.923077   
1612100            4.000000       NaN       NaN                   NaN   
1647190                 NaN       NaN       NaN              4.000000   
1653100                 NaN       NaN       NaN                   NaN   
1737200                 NaN       NaN  4.000000                   NaN   
1837210                 NaN       NaN       NaN                   NaN   
1837330                 NaN       NaN       NaN                   NaN   
1875060            4.391304       NaN  3.000000              4.375000   
1919350                 NaN       NaN       NaN                   NaN   
1956670                 NaN       NaN       NaN                   NaN   
2109270            4.200000       NaN  4.000000                   NaN   
2211500                 NaN       NaN       NaN                   NaN   
222730             4.333333       NaN  3.363636              3.200000   
2289390                 NaN       NaN       NaN                   NaN   
2294910                 NaN       NaN       NaN                   NaN   
2300300            4.428571       NaN       NaN                   NaN   
244710                  NaN  3.250000  3.000000              2.500000   
2576600                 NaN       NaN       NaN                   NaN   
2594480                 NaN  4.000000       NaN                   NaN   
2634610                 NaN       NaN       NaN                   NaN   
2635350                 NaN       NaN  3.000000              5.000000   
275100                  NaN  3.000000  4.000000              3.000000   
2878100            3.000000       NaN       NaN                   NaN   
3059480                 NaN       NaN       NaN                   NaN   
382310                  NaN  4.153846  4.192308              3.784314   
431180                  NaN       NaN       NaN                   NaN   
434000                  NaN       NaN       NaN                   NaN   
486310                  NaN  1.000000  4.000000              1.000000   
512460                  NaN  3.000000  3.000000                   NaN   
853550                  NaN       NaN  3.928571              4.285714   
898890                  NaN  4.000000       NaN              3.000000   
914750             3.000000       NaN       NaN                   NaN   
916900                  NaN       NaN       NaN                   NaN   
926990                  NaN  5.000000  2.000000              5.000000   

category                                                             \
query    Combat System and Action Economy and Management Simulation   
game                                                                  
1028590                       NaN                    NaN   4.000000   
1062090                  3.000000               3.987179   4.000000   
1133120                       NaN                    NaN   4.500000   
1346210                       NaN                    NaN        NaN  